# DuckLake writer against local MinIO

This notebook targets a locally running MinIO (or RustFS) endpoint and writes the e-commerce dataset via DuckLake (DuckDB extension). Follow the instructions in `examples/README.md` to start MinIO and create a bucket (e.g. `demo`).

## Configuration

Adjust the endpoint and credentials below as needed. The defaults match the docker-compose setup (`minioadmin/minioadmin`).

In [1]:
import datetime
from pathlib import Path
import duckdb
from dataset_generator import create_generator, create_writer, WriterOptions, write_dataset, S3Config, CatalogConfig
endpoint = "http://localhost:9000"
access_key = "demo"
secret_key = "demo1234"
bucket = "demo"
prefix = "ducklake"
catalog_db = Path("examples/demo_output/ducklake/catalog.db").resolve()
catalog_db.parent.mkdir(parents=True, exist_ok=True)

## Generate and write

This creates a small dataset and writes partitioned Parquet files plus DuckLake metadata into MinIO.

In [3]:
generator = create_generator("ecommerce",
    seed=21,
    n_customers=334198,
    n_products=123456,
    orders_per_day=10341,
    order_items_mean=5.0,
    start_date=datetime.date(2023, 4, 1),
    end_date=datetime.date(2025, 4, 1),
    file_rows_target=150_000,
)
s3_config = S3Config(uri=f"s3://{bucket}", key=access_key, secret=secret_key, endpoint_url=endpoint, region="us-east-1", use_ssl=False)
catalog = CatalogConfig(kind="sqlite", uri=f"sqlite:///{catalog_db}")
writer = create_writer("ducklake", output_uri=f"{bucket}/{prefix}", s3=s3_config, catalog=catalog, options=WriterOptions(file_rows_target=150))
write_dataset(generator, writer)

## Inspect data with DuckDB

Use DuckDB to browse the table through the DuckLake extension.

In [ ]:
con = duckdb.connect()
con.execute("INSTALL ducklake")
con.execute("LOAD ducklake")
con.execute("SET s3_endpoint = ?", [endpoint])
con.execute("SET s3_access_key_id = ?", [access_key])
con.execute("SET s3_secret_access_key = ?", [secret_key])
con.execute("SET s3_use_ssl = 0")
con.sql(f"SELECT * FROM ducklake."{bucket}"."{prefix}/orders" LIMIT 5")